In [3]:
cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [30]:
!pip install python-whois

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103246 sha256=3ef8bbaeb9247eab130b541c836ff685e9cb36c897a3564ee425a2b022a61bf1
  Stored in directory: /root/.cache/pip/wheels/10/f1/87/145023b9a206e2e948be6480c61ef3fd3dbb81ef11b6977782
Successfully built python-whois


In [4]:
import pandas as pd
import numpy as np

In [5]:
import urllib.request
import os
import ssl, socket
from datetime import datetime
import whois

import json

  Preparing metadata (setup.py) ... done
  Created wheel for whois: filename=whois-0.9.27-py3-none-any.whl size=30469 sha256=1fe9f852df69b83d2416743f607c4b0c239e06b6e3da59661a8fe6820ea252b8
  Stored in directory: /root/.cache/pip/wheels/71/17/36/d62eb5bcc416650499a7259d584c11e4a778de5ce0e72a8dbf
Successfully built whois


In [6]:
import json

In [7]:
from tensorflow.keras.preprocessing import sequence
max_review_length = 60

In [8]:
from keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('modelNew91_59.h5')

In [9]:
charMap={}
with open('charencode.json') as json_file:
    charMap = json.load(json_file)


In [10]:
# Convert and write JSON object to file
sslMap={}
with open("data/sslIssuers.json") as outfile:
   sslMap = json.load(outfile)

In [11]:
charDict={}
def asciize(strIn):
    global charMap
    charList=[]
    #print(strIn)
    for ch in str(strIn):
      charList.append(charMap[ch])
    return charList

In [12]:
def getDomainName(url):
    dom=url
    if "." in url:
            urlList=url.split(".")
            ln=len(urlList)
            if ln==2:
                dom=urlList[0]
                return dom
            elif ln==3:
                dom=urlList[0]
                return dom
            elif ln==4:
                dom=urlList[1]
                return dom
            elif not dom.isnumeric():
                return dom

    else:
        return dom

In [13]:
def predictUrl(url):
  url=getDomainName(url)
  arr = [np.array(asciize(url))]
  #print(arr)
  my_test = sequence.pad_sequences(arr, maxlen=60, padding='post',truncating='post')
  testScore=model.predict(my_test)
  return testScore[0][0]

In [14]:
predictUrl("karevakfi.com")

1/1 [==============================] - 1s 1s/step


0.11185269

In [15]:
def getSslCertDetails(hostname):
    try:
      ctx = ssl.create_default_context()
      with ctx.wrap_socket(socket.socket(), server_hostname=hostname) as s:
          s.connect((hostname, 443))
          cert = s.getpeercert()

      dateTimeStr="%b %d %H:%M:%S %Y %Z"
      certStart=cert["notBefore"]
      certEnd=cert["notAfter"]
      certEnd=datetime.strptime(certEnd,dateTimeStr).date()
      certStart=datetime.strptime(certStart,dateTimeStr).date()
      current_dateTime = datetime.now().date()
      willEnd=(certEnd - current_dateTime).days
      startedBefore=(current_dateTime-certStart).days
      subject = dict(x[0] for x in cert['subject'])
      issued_to = subject['commonName']
      issuer = dict(x[0] for x in cert['issuer'])
      issued_by = issuer['organizationName']
      return willEnd,startedBefore,issued_by
    except:
        return None

In [16]:
def whoisCheck(url):
    try:
      whoIs=whois.whois(url)
      urlDict={}
      if whoIs['domain_name'] is not None:
        current_dateTime = datetime.now()
        dateTimeStr="%Y-%m-%d %H:%M:%S"
        current_dateTime = datetime.now()
        #print(whoIs['creation_date'])
        if type(whoIs['domain_name']) == list:
          whoIs['domain_name']=whoIs['domain_name'][1]
        urlDict['domain_name']=whoIs['domain_name']
        if isinstance(whoIs['creation_date'], datetime):
          urlDict['domain_start']=(current_dateTime-whoIs['creation_date']).days
          urlDict['domain_end']=(whoIs['expiration_date']-current_dateTime).days
        elif type(whoIs['creation_date']) == list:
          urlDict['domain_start']=(current_dateTime-whoIs['creation_date'][0]).days
          urlDict['domain_end']=(whoIs['expiration_date'][0]-current_dateTime).days
        else:
          urlDict['domain_start']=datetime.strptime(whoIs['creation_date'],dateTimeStr).date()
          urlDict['domain_end']=datetime.strptime(whoIs['expiration_date'],dateTimeStr).date()
          urlDict['domain_start']=(current_dateTime-urlDict['domain_start']).days
          urlDict['domain_end']=(current_dateTime-urlDict['domain_end']).days
        if "city" in whoIs:
          urlDict['domain_city']=whoIs['city']
          urlDict['domain_state']=whoIs['state']
          urlDict['domain_country']=whoIs['country']
        if "domain_registrar" in whoIs:
          urlDict['domain_registrar']=whoIs['registrar']
        return urlDict['domain_start'],urlDict['domain_end']
    except:
        return None

In [26]:
def getPh2Data(url):
  ph1Score=predictUrl(url)
  myDict={}
  dmStart=-1
  dmEnd=-1
  myUrl=url
  if (whoisCheck(url) is not None):
      dmStart,dmEnd=whoisCheck(url)

  if (getSslCertDetails(url) is not None):
      sslEnd,sslStart,sslIssuer=getSslCertDetails(url)
  else:
      sslStart=-1
      sslEnd=-1
      sslIssuer=None
  if "." in url:
      urlList=url.split(".")
      ln=len(urlList)
      if ln==2:
          dom=urlList[0]
      if ln==3:
          dom=urlList[0]
      if ln==4:
          dom=urlList[1]
      #if not dom.isnumeric():
      myDict = {'url' : myUrl, 'domainStart' : dmStart, 'domainEnd' : dmEnd,'sslStart':sslStart,'sslEnd':sslEnd,'sslIssuer':sslIssuer,'ph1Score':ph1Score}
  return myDict


In [27]:
getPh2Data("acikdeniz.denizle100yilbasvuru.online")

1/1 [==============================] - 0s 19ms/step


{'url': 'acikdeniz.denizle100yilbasvuru.online',
 'domainStart': 1,
 'domainEnd': 365,
 'sslStart': -1,
 'sslEnd': -1,
 'sslIssuer': None,
 'ph1Score': 0.83700734}

In [ ]:
#load all legal sites for phase2
legal2Df = pd.read_csv('data/legalPhase2.csv')

In [ ]:
legal2Df.columns

Index(['Unnamed: 0', 'url', 'domainStart', 'domainEnd', 'sslStart', 'sslEnd',
       'sslIssuer', 'label'],
      dtype='object')

In [ ]:
#delete first column came from file as index
legal2Df.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
#set all labels to 0 for legal sites
legal2Df['label']=0

In [ ]:
legal2Df.head()

,url,domainStart,domainEnd,sslStart,sslEnd,sslIssuer,label
0,bosnayapi,3531,1215,49,41,Google Trust Services LLC,0
1,fizyopt,1024,69,23,67,Let's Encrypt,0
2,franke,8618,146,18,72,Let's Encrypt,0
3,e-yonetim,4859,253,4,86,Let's Encrypt,0
4,mescainsaat,1633,192,39,51,Let's Encrypt,0


In [ ]:
len(legal2Df)

20778

In [ ]:
#load all phishing sites for phase2
phishing2Df = pd.read_csv('data/phishingPhase2.csv')

In [ ]:
phishing2Df.head()

,url,domainStart,domainEnd,sslStart,sslEnd,sslIssuer,label
0,101efsanekasimindirimleri-online-alisveris-oze...,0,365,-1,-1,NaN,1
1,sahibindan-param-guvende,0,365,1,89,Let's Encrypt,1
2,ozel-firsatlar-kasim-online,2,363,-1,-1,NaN,1
3,efsanekasimindirimleri-online-alisveris-ozel-i...,0,365,-1,-1,NaN,1
4,efsanekasimindirimleri-online-alisveris-ozel-i...,0,365,-1,-1,NaN,1


In [ ]:
len(phishing2Df)

24506

In [ ]:
ph2TrainDf=legal2Df.append(phishing2Df,ignore_index=True)

<ipython-input-23-164936e68a7e>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ph2TrainDf=legal2Df.append(phishing2Df,ignore_index=True)


In [ ]:
ph2TrainDf.head()

,url,domainStart,domainEnd,sslStart,sslEnd,sslIssuer,label
0,bosnayapi,3531,1215,49,41,Google Trust Services LLC,0
1,fizyopt,1024,69,23,67,Let's Encrypt,0
2,franke,8618,146,18,72,Let's Encrypt,0
3,e-yonetim,4859,253,4,86,Let's Encrypt,0
4,mescainsaat,1633,192,39,51,Let's Encrypt,0


In [ ]:
len(ph2TrainDf)

45284

In [ ]:
ph2TrainDf=pd.read_csv("data/phase2DataAll.csv")

In [ ]:
ph2TrainDf.head()

,url,domainStart,domainEnd,sslStart,sslEnd,sslIssuer,label
0,bosnayapi,3531,1215,49,41,Google Trust Services LLC,0
1,fizyopt,1024,69,23,67,Let's Encrypt,0
2,franke,8618,146,18,72,Let's Encrypt,0
3,e-yonetim,4859,253,4,86,Let's Encrypt,0
4,mescainsaat,1633,192,39,51,Let's Encrypt,0


In [ ]:
#ph2TrainDf = ph2TrainDf.sample(frac=1).reset_index(drop=True)
#ph2TrainDf = ph2TrainDf.sample(frac=1).reset_index(drop=True)

In [ ]:
def getMatches(list1,list2):
  t=0
  mCnt=0
  umCnt=0
  for item in list1:
    if item==list2[t]:
      mCnt+=1
    else:
      umCnt+=1
    t+=1
  return mCnt,umCnt

In [ ]:
#Phishing scores from deep learning model will be kept in this list
ph1PhScrList=[]
#labels according to phishing score from deep learning will be kept in this list
ph1LbList=[]
ph1Url=[]
processedDf=pd.read_csv("data/phase2Tmp.csv")
ph1PhScrList=list(processedDf['scores'].values)
ph1LbList=list(processedDf['label'].values)
ph1Url=list(processedDf['url'].values)

In [ ]:
type(ph1PhScrList)

list

In [ ]:
len(ph1Url)

40000

In [ ]:
t=0
prCnt=1000
for url in ph2TrainDf["url"].values:
  if t<40000:
    t+=1
    continue
  #if url in ph1Url:
    #t+=1
    #continue
  if (t%prCnt==0) and t>0:
    print(f"++++++{t}+++++++")
    m,um=getMatches(ph1LbList[-prCnt:],ph2TrainDf["label"].values[(t-prCnt):t])
    print(f"{ph1LbList[-prCnt:]}---{ph2TrainDf['label'].values[(t-prCnt):t]}++++++match:{m}+++++++Unmatch:{um}")
  scr=predictUrl(url)
  t+=1
  ph1PhScrList.append(scr)
  ph1Url.append(url)
  if scr>0.5:
    ph1LbList.append(1);
  else:
    ph1LbList.append(0);
  if t>50000 and (t%2500==0):
    dfTemp = pd.DataFrame(ph1PhScrList,columns =['scores'])
    dfTemp["url"]=ph1Url
    dfTemp["label"]=ph1LbList
    dfTemp.to_csv("data/phase2Tmp.csv",index=False)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
1/1 [==============================] - 0s 26ms/step
++++++41000+++++++
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,

In [ ]:
dfTemp = pd.DataFrame(ph1PhScrList,columns =['scores'])
dfTemp["url"]=ph1Url
dfTemp["label"]=ph1LbList
dfTemp.to_csv("data/phase2Scores.csv",index=False)

In [ ]:
len(ph2TrainDf)

45284

In [ ]:
ph2TrainDf["dlPhishingScore"]=ph1PhScrList
ph2TrainDf.to_csv("data/phase2DataAllwithps.csv",index=False)

In [ ]:
def testUrlList(urlList):
  testScores=[]
  for url in urlList:




In [ ]:
testUrlList=ph2TrainDf.head(500)["url"].values
testScores=[]
for url in
if scr>0.5:
    ph1LbList.append(1)
  else:
    ph1LbList.append(0)

In [ ]:
predictUrl("mescainsaat")

1/1 [==============================] - 0s 25ms/step


0.9920838